# Ensembles & Pipelines
>  Finally you'll learn how to make your models more efficient. You'll find out how to use pipelines to make your code clearer and easier to maintain. Then you'll use cross-validation to better test your models and select good model parameters. Finally you'll dabble in two types of ensemble model.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 4 exercises "Machine Learning with PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [ ]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/22-machine-learning-with-pyspark/datasets/flights.csv
spark = SparkSession.builder.master('local[*]').appName('flights').getOrCreate()

In [ ]:
flights = spark.read.csv('flights.csv', sep=',', header=True, inferSchema=True, nullValue='NA')
print("The data contain %d records." % flights.count())
flights.show(5)
print(flights.printSchema())
print(flights.dtypes)

The data contain 50000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)

None
[('mon', 'int'), 

## Pipeline

### Flight duration model: Pipeline stages

<p>You're going to create the stages for the flights duration model pipeline. You will use these in the next exercise to build a pipeline and to create a regression model.</p>

In [ ]:
from pyspark.sql.functions import round
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
flights = flights.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')

Instructions
<ul>
<li>Create an indexer to convert the 'org' column into an indexed column called 'org_idx'.</li>
<li>Create a one-hot encoder to convert the 'org_idx' and 'dow' columns into dummy variable columns called 'org_dummy' and 'dow_dummy'.</li>
<li>Create an assembler which will combine the 'km' column with the two dummy variable columns. The output column should be called 'features'.</li>
<li>Create a linear regression object to predict flight duration.</li>
</ul>

In [ ]:
# Convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoder(
    inputCols=['org_idx', 'dow'],
    outputCols=['org_dummy', 'dow_dummy']
)

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

# A linear regression object
regression = LinearRegression(labelCol='duration')

### Flight duration model: Pipeline model

<div class=""><p>You're now ready to put those stages together in a pipeline.</p>
<p>You'll construct the pipeline and then train the pipeline on the training data. This will apply each of the individual stages in the pipeline to the training data in turn. None of the stages will be exposed to the testing data at all: there will be no leakage!</p>
<p>Once the entire pipeline has been trained it will then be used to make predictions on the testing data.</p>
<p>The data are available as <code>flights</code>, which has been randomly split into <code>flights_train</code> and <code>flights_test</code>.</p></div>

In [ ]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

Instructions
<ul>
<li>Import the class for creating a pipeline.</li>
<li>Create a pipeline object and specify the <code>indexer</code>, <code>onehot</code>, <code>assembler</code> and <code>regression</code> stages, in this order.</li>
<li>Train the pipeline on the training data.</li>
<li>Make predictions on the testing data.</li>
</ul>

In [ ]:
# Import class for creating a pipeline
from pyspark.ml import Pipeline

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

# Train the pipeline on the training data
pipeline = pipeline.fit(flights_train)

# Make predictions on the testing data
predictions = pipeline.transform(flights_test)

**A pipeline makes your code easier to read and maintain.**

### SMS spam pipeline

<div class=""><p>You haven't looked at the SMS data for quite a while. Last time we did the following:</p>
<ul>
<li>split the text into tokens</li>
<li>removed stop words</li>
<li>applied the hashing trick</li>
<li>converted the data from counts to IDF and</li>
<li>trained a logistic regression model.</li>
</ul>
<p>Each of these steps was done independently. This seems like a great application for a pipeline!</p></div>

In [ ]:
%%capture
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/22-machine-learning-with-pyspark/datasets/sms.csv
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])
sms = spark.read.csv('sms.csv', sep=';', header=False, schema=schema,  nullValue='NA')

Instructions
<ul>
<li>Create an object for splitting text into tokens.</li>
<li>Create an object to remove stop words. Rather than explicitly giving the input column name, use the <code>getOutputCol()</code> method on the previous object.</li>
<li>Create objects for applying the hashing trick and transforming the data into a TF-IDF. Use the <code>getOutputCol()</code> method again.</li>
<li>Create a pipeline which wraps all of the above steps as well as an object to create a Logistic Regression model.</li>
</ul>

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

# Break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')

# Remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='terms')

# Apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol='terms', outputCol="hash")
idf = IDF(inputCol='hash', outputCol="features")

# Create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression()
pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

## Cross-Validation

### Cross validating simple flight duration model

<div class=""><p>You've already built a few models for predicting flight duration and evaluated them with a simple train/test split. However, cross-validation provides a much better way to evaluate model performance.</p>
<p>In this exercise you're going to train a simple model for flight duration using cross-validation. Travel time is usually strongly correlated with distance, so using the <code>km</code> column alone should give a decent model.</p>
<p>The data have been randomly split into <code>flights_train</code> and <code>flights_test</code>.</p>
<p>The following classes have already been imported: <code>LinearRegression</code>, <code>RegressionEvaluator</code>, <code>ParamGridBuilder</code> and <code>CrossValidator</code>.</p></div>

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
assembler = VectorAssembler(inputCols=['km'], outputCol='features')
flights = assembler.transform(flights.drop('features'))
flights.show(5)
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

+---+---+---+-------+------+---+------+--------+-----+------+--------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|features|
+---+---+---+-------+------+---+------+--------+-----+------+--------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|3465.0|[3465.0]|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0| [509.0]|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8| 542.0| [542.0]|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|1989.0|[1989.0]|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null| 415.0| [415.0]|
+---+---+---+-------+------+---+------+--------+-----+------+--------+
only showing top 5 rows



Instructions
<ul>
<li>Create an empty parameter grid.</li>
<li>Create objects for building and evaluating a linear regression model. The model should predict the "duration" field.</li>
<li>Create a cross-validator object. Provide values for the <code>estimator</code>, <code>estimatorParamMaps</code> and <code>evaluator</code> arguments. Choose 5-fold cross validation.</li>
<li>Train and test the model across multiple folds of the training data.</li>
</ul>

In [ ]:
# Create an empty parameter grid
params = ParamGridBuilder().build()

# Create objects for building and evaluating a regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

# Create a cross validator
cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

# Train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

# NOTE: Since cross-valdiation builds multiple models, the fit() method can take a little while to complete.

### Cross validating flight duration model pipeline

<div class=""><p>The cross-validated model that you just built was simple, using <code>km</code> alone to predict <code>duration</code>.</p>
<p>Another important predictor of flight duration is the origin airport. Flights generally take longer to get into the air from busy airports. Let's see if adding this predictor improves the model!</p>
<p>In this exercise you'll add the <code>org</code> field to the model. However, since <code>org</code> is categorical, there's more work to be done before it can be included: it must first be transformed to an index and then one-hot encoded before being assembled with <code>km</code> and used to build the regression model. We'll wrap these operations up in a pipeline.</p>
<p>The following objects have already been created:</p>
<ul>
<li><code>params</code> — an empty parameter grid</li>
<li><code>evaluator</code> — a regression evaluator</li>
<li><code>regression</code> — a <code>LinearRegression</code> object with <code>labelCol='duration'</code>.</li>
</ul>
<p>All of the required classes have already been imported.</p></div>

In [ ]:
# Create an empty paramter grid
params = ParamGridBuilder().build()

# Create regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

Instructions
<ul>
<li>Create a string indexer. Specify the input and output fields as <code>org</code> and <code>org_idx</code>.</li>
<li>Create a one-hot encoder. Name the output field <code>org_dummy</code>.</li>
<li>Assemble the <code>km</code> and <code>org_dummy</code> fields into a single field called <code>features</code>.</li>
<li>Create a pipeline using the following operations: string indexer, one-hot encoder, assembler and linear regression. Use this to create a cross-validator.</li>
</ul>

In [ ]:
# Create an indexer for the org field
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# Create an one-hot encoder for the indexed org field
onehot = OneHotEncoder(
    inputCols=['org_idx'],
    outputCols=['org_dummy']
)

# Assemble the km and one-hot encoded fields
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Create a pipeline and cross-validator.
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator=pipeline,
          estimatorParamMaps=params,
          evaluator=evaluator)

**Wrapping operations in a pipeline makes cross validating the entire workflow easy!**

## Grid Search

### Optimizing flights linear regression

<div class=""><p>Up until now you've been using the default hyper-parameters when building your models. In this exercise you'll use cross validation to choose an optimal (or close to optimal) set of model hyper-parameters.</p>
<p>The following have already been created:</p>
<ul>
<li><code>regression</code> — a <code>LinearRegression</code> object</li>
<li><code>pipeline</code> — a pipeline with string indexer, one-hot encoder, vector assembler and linear regression and</li>
<li><code>evaluator</code> — a <code>RegressionEvaluator</code> object.</li>
</ul></div>

Instructions
<ul>
<li>Create a parameter grid builder.</li>
<li>Add grids for with <code>regression.regParam</code> (values 0.01, 0.1, 1.0, and 10.0) and <code>regression.elasticNetParam</code> (values 0.0, 0.5, and 1.0).</li>
<li>Build the grid.</li>
<li>Create a cross validator, specifying five folds.</li>
</ul>

In [ ]:
# Create parameter grid
params = ParamGridBuilder()

# Add grids for two parameters
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0,10.0]) \
               .addGrid(regression.elasticNetParam, [ 0.0, 0.5, 1.0])

# Build the parameter grid
params = params.build()
print('Number of models to be tested: ', len(params))

# Create cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

Number of models to be tested:  12


### Dissecting the best flight duration model

<div class=""><p>You just set up a <code>CrossValidator</code> to find good parameters for the linear regression model predicting flight duration.</p>
<p>Now you're going to take a closer look at the resulting model, split out the stages and use it to make predictions on the testing data.</p>
<p>The following have already been created:</p>
<ul>
<li><code>cv</code> — a trained <code>CrossValidatorModel</code> object and</li>
<li><code>evaluator</code> — a <code>RegressionEvaluator</code> object.</li>
</ul>
<p>The flights data have been randomly split into <code>flights_train</code> and <code>flights_test</code>.</p></div>

In [ ]:
flights_train, flights_test = flights.drop('features').randomSplit([0.8, 0.2])
cv = cv.fit(flights_train)

Instructions
<ul>
<li>Retrieve the best model.</li>
<li>Look at the stages in the best model.</li>
<li>Isolate the linear regression stage and extract its parameters.</li>
<li>Use the best model to generate predictions on the testing data and calculate the RMSE.</li>
</ul>

In [ ]:
# Get the best model from cross validation
best_model = cv.bestModel

# Look at the stages in the best model
print(best_model.stages)

# Get the parameters for the LinearRegression object in the best model
best_model.stages[3].extractParamMap()

# Generate predictions on testing data using the best model then calculate RMSE
predictions = best_model.transform(flights_test)
evaluator.evaluate(predictions)

[StringIndexerModel: uid=StringIndexer_dcf9b7796637, handleInvalid=error, OneHotEncoderModel: uid=OneHotEncoder_36d0a2f51de2, dropLast=true, handleInvalid=error, numInputCols=1, numOutputCols=1, VectorAssembler_d6bd76fba3fa, LinearRegressionModel: uid=LinearRegression_d7b047b1125a, numFeatures=8]


11.333854388909936

### SMS spam optimised

<div class=""><p>The pipeline you built earlier for the SMS spam model used the default parameters for all of the elements in the pipeline. It's very unlikely that these parameters will give a particularly good model though.</p>
<p>In this exercise you'll set up a parameter grid which can be used with cross validation to choose a good set of parameters for the SMS spam classifier.</p>
<p>The following are already defined:</p>
<ul>
<li><code>hasher</code> — a <code>HashingTF</code> object and</li>
<li><code>logistic</code> —  a <code>LogisticRegression</code> object.</li>
</ul></div>

Instructions
<ul>
<li>Create a parameter grid builder object.</li>
<li>Add grid points for <code>numFeatures</code> and <code>binary</code> parameters to the <code>HashingTF</code> object, giving values 1024, 4096 and 16384, and True and False, respectively.</li>
<li>Add grid points for <code>regParam</code> and <code>elasticNetParam</code> parameters to the <code>LogisticRegression</code> object, giving values of 0.01, 0.1, 1.0 and 10.0, and 0.0, 0.5, and 1.0 respectively.</li>
<li>Build the parameter grid.</li>
</ul>

In [ ]:
# Create parameter grid
params = ParamGridBuilder()

# Add grid for hashing trick parameters
params = params.addGrid(hasher.numFeatures, [1024, 4096, 16384]) \
               .addGrid(hasher.binary, [True, False])

# Add grid for logistic regression parameters
params = params.addGrid(logistic.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(logistic.elasticNetParam, [0.0, 0.5, 1.0])

# Build parameter grid
params = params.build()

**Using cross-validation on a pipeline makes it possible to optimise each stage in the workflow.**

### How many models for grid search?

<div class=""><p>How many models will be built when the cross-validator below is fit to data?</p>
<pre><code>params = ParamGridBuilder().addGrid(hasher.numFeatures, [1024, 4096, 16384]) \
                           .addGrid(hasher.binary, [True, False]) \
                           .addGrid(logistic.regParam, [0.01, 0.1, 1.0, 10.0]) \
                           .addGrid(logistic.elasticNetParam, [0.0, 0.5, 1.0]) \
                           .build()

cv = CrossValidator(..., estimatorParamMaps=params, numFolds=5)
</code></pre></div>

<pre>
Possible Answers
3
5
72
<b>360</b>
</pre>

In [ ]:
3*2*4*3*5

360

**There are 72 points in the parameter grid and 5 folds in the cross-validator. The product is 360. It takes time to build all of those models, which is why we're not doing it here!**

## Ensemble

### Delayed flights with Gradient-Boosted Trees

<div class=""><p>You've previously built a classifier for flights likely to be delayed using a Decision Tree. In this exercise you'll compare a Decision Tree model to a Gradient-Boosted Trees model.</p>
<p>The flights data have been randomly split into <code>flights_train</code> and <code>flights_test</code>.</p></div>

In [ ]:
#from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
assembler = VectorAssembler(inputCols=['mon', 'depart', 'duration'], outputCol='features')
flights = assembler.transform(flights.drop('features'))
flights = flights.withColumn('label', (flights.delay >= 15).cast('integer'))
flights = flights.select('mon', 'depart', 'duration', 'features', 'label')
flights = flights.dropna()

flights.show(5)

+---+------+--------+-----------------+-----+
|mon|depart|duration|         features|label|
+---+------+--------+-----------------+-----+
|  0| 16.33|      82| [0.0,16.33,82.0]|    1|
|  2|  6.17|      82|  [2.0,6.17,82.0]|    0|
|  9| 10.33|     195|[9.0,10.33,195.0]|    0|
|  5|  7.98|     102| [5.0,7.98,102.0]|    0|
|  7| 10.83|     135|[7.0,10.83,135.0]|    1|
+---+------+--------+-----------------+-----+
only showing top 5 rows



In [ ]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

Instructions
<ul>
<li>Import the classes required to create Decision Tree and Gradient-Boosted Tree classifiers.</li>
<li>Create Decision Tree and Gradient-Boosted Tree classifiers. Train on the training data.</li>
<li>Create an evaluator and calculate AUC on testing data for both classifiers. Which model performs better?</li>
<li>Find the number of trees and the relative importance of features in the Gradient-Boosted Tree classifier.</li>
</ul>

In [ ]:
# Import the classes required
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create model objects and train on training data
tree = DecisionTreeClassifier().fit(flights_train)
gbt = GBTClassifier(maxIter=5).fit(flights_train)

# Compare AUC on test data
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(tree.transform(flights_test))
evaluator.evaluate(gbt.transform(flights_test))

0.6729637463929005

In [ ]:
# Find the number of trees and the relative importance of features
gbt.trees

[DecisionTreeRegressionModel: uid=dtr_11eb0f50ff25, depth=5, numNodes=63, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_e1f30df1158e, depth=5, numNodes=63, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_fa0736d1357c, depth=5, numNodes=63, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_354745ea84dc, depth=5, numNodes=63, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_3c7278ec6db8, depth=5, numNodes=61, numFeatures=3]

In [ ]:
gbt.featureImportances

SparseVector(3, {0: 0.4241, 1: 0.3165, 2: 0.2595})

**A Gradient-Boosted Tree almost always provides better performance than a plain Decision Tree.**

### Delayed flights with a Random Forest

<div class=""><p>In this exercise you'll bring together cross validation and ensemble methods. You'll be training a Random Forest classifier to predict delayed flights, using cross validation to choose the best values for model parameters.</p>
<p>You'll find good values for the following parameters:</p>
<ul>
<li><code>featureSubsetStrategy</code> — the number of features to consider for splitting at each node and</li>
<li><code>maxDepth</code> — the maximum number of splits along any branch.</li>
</ul>
<p>Unfortunately building this model takes too long, so we won't be running the <code>.fit()</code> method on the pipeline.</p></div>

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

Instructions
<ul>
<li>Create a random forest classifier object.</li>
<li>Create a parameter grid builder object. Add grid points for the <code>featureSubsetStrategy</code> and <code>maxDepth</code> parameters.</li>
<li>Create binary classification evaluator.</li>
<li>Create a cross-validator object, specifying the estimator, parameter grid and evaluator. Choose 5-fold cross validation.</li>
</ul>

In [ ]:
# Create a random forest classifier
forest = RandomForestClassifier()

# Create a parameter grid
params = ParamGridBuilder() \
            .addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
            .addGrid(forest.maxDepth, [2, 5, 10]) \
            .build()

# Create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()

# Create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

### Evaluating Random Forest

<div class=""><p>In this final exercise you'll be evaluating the results of cross-validation on a Random Forest model. </p>
<p>The following have already been created:</p>
<ul>
<li><code>cv</code> - a cross-validator which has already been fit to the training data</li>
<li><code>evaluator</code> — a <code>BinaryClassificationEvaluator</code> object and</li>
<li><code>flights_test</code> — the testing data.</li>
</ul></div>

In [ ]:
cv = cv.fit(flights_train)

Instructions
<ul>
<li>Retrieve a list of average AUC metrics across all models in the parameter grid.</li>
<li>What is the average AUC for the best model? This will be the <em>largest</em> AUC in the list.</li>
<li>Find the value of the <code>maxDepth</code> and <code>featureSubsetStrategy</code> parameters for the best model.</li>
<li>Calculate the AUC for the best model predictions on the testing data.</li>
</ul>

In [ ]:
# Average AUC for each parameter combination in grid
avg_auc = cv.avgMetrics

# Average AUC for the best model
best_model_auc =  max(avg_auc)

# What's the optimal parameter value?
opt_max_depth = cv.bestModel.explainParam('maxDepth')
opt_feat_substrat = cv.bestModel.explainParam('featureSubsetStrategy')

# AUC for best model on testing data
best_auc = evaluator.evaluate(cv.transform(flights_test))

In [ ]:
best_auc

0.6786894272626705